<a href="https://colab.research.google.com/github/louistrue/learn-ifc-bfh25-D/blob/main/BFH-25-PropertySets-Dashboard.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# 🧱 IFC PropertySets Export to CSV

Extrahiert alle PropertySets aus einem IFC-Modell und speichert sie als CSV.

**Outputs:**
- 📄 `property_sets_detailed.csv` - Alle Properties inkl. Element-Details
- 📄 `property_sets_summary.csv` - PropertySet-Property-Kombinationen

In [ ]:
%pip install ifcopenshell


Note: you may need to restart the kernel to use updated packages.


In [ ]:
import ifcopenshell
import ifcopenshell.util.element as uel
import pandas as pd
from collections import defaultdict


In [ ]:
# 📂 IFC-Modell laden
# Für Colab: Datei hochladen
# Für lokal: Pfad anpassen
try:
    from google.colab import files
    print("🔼 Google Colab erkannt - Bitte IFC-Datei hochladen...")
    uploaded = files.upload()
    IFC_FILE = list(uploaded.keys())[0]
    print(f"✅ Datei hochgeladen: {IFC_FILE}")
except:
    # Lokaler Pfad
    IFC_FILE = "Modelle/BFH-25/02_BIMcollab_Example_ARC.ifc"
    print(f"💻 Lokaler Modus - Lade: {IFC_FILE}")

model = ifcopenshell.open(IFC_FILE)
print(f"✅ Modell geladen: {model.schema} | {len(model.by_type('IfcElement'))} Elemente")


💻 Lokaler Modus - Lade: Modelle/BFH-25/02_BIMcollab_Example_ARC.ifc


✅ Modell geladen: IFC2X3 | 1354 Elemente


In [ ]:
# 📊 PropertySets extrahieren
detailed_data = []
summary_data = defaultdict(set)

print("🔍 Extrahiere PropertySets...")

for element in model.by_type('IfcElement'):
    element_id = element.GlobalId
    element_type = element.is_a()
    element_name = getattr(element, 'Name', None) or 'Unnamed'
    
    # PropertySets abrufen
    psets = uel.get_psets(element)
    
    for pset_name, properties in psets.items():
        if not isinstance(properties, dict):
            continue
        
        for prop_name, prop_value in properties.items():
            if prop_name == 'id':
                continue
            
            # Detaillierte Daten
            detailed_data.append({
                'Element_GlobalId': element_id,
                'Element_Class': element_type,
                'Element_Name': element_name,
                'PropertySet': pset_name,
                'Property': prop_name,
                'Value': str(prop_value)
            })
            
            # Summary
            summary_data[pset_name].add(prop_name)

print(f"✅ {len(detailed_data)} Property-Werte extrahiert")
print(f"✅ {len(summary_data)} PropertySets gefunden")


🔍 Extrahiere PropertySets...
✅ 11254 Property-Werte extrahiert
✅ 22 PropertySets gefunden


In [ ]:
# 💾 CSV Export: Detaillierte Daten
if detailed_data:
    df_detailed = pd.DataFrame(detailed_data)
    output_file = 'property_sets_detailed.csv'
    df_detailed.to_csv(output_file, index=False, encoding='utf-8-sig')
    
    print(f"✅ CSV gespeichert: {output_file}")
    print(f"   Zeilen: {len(df_detailed):,}")
    display(df_detailed.head(10))
    
    # Download in Colab
    try:
        from google.colab import files
        files.download(output_file)
        print(f"📥 Download gestartet: {output_file}")
    except:
        pass


✅ CSV gespeichert: property_sets_detailed.csv
   Zeilen: 11,254


,Element_GlobalId,Element_Type,Element_Name,PropertySet,Property,Value
0,11EjrZoVgVGQm8NAB8Ffig,IfcBeam,Beam rafter,Pset_BeamCommon,Reference,Rafters 71 x 171
1,11EjrZoVgVGQm8NAB8Ffig,IfcBeam,Beam rafter,Pset_BeamCommon,FireRating,R 90
2,11EjrZoVgVGQm8NAB8Ffig,IfcBeam,Beam rafter,Pset_BeamCommon,LoadBearing,True
3,11EjrZoVgVGQm8NAB8Ffig,IfcBeam,Beam rafter,Pset_BeamCommon,IsExternal,False
4,11EjrZoVgVGQm8NAB8Ffig,IfcBeam,Beam rafter,Pset_FireRatingProperties,FireResistanceRating,60
5,11EjrZoVgVGQm8NAB8Ffig,IfcBeam,Beam rafter,BaseQuantities,NetSurfaceAreaExtrudedSides,4.46248
6,11EjrZoVgVGQm8NAB8Ffig,IfcBeam,Beam rafter,BaseQuantities,Length,9220.0
7,11EjrZoVgVGQm8NAB8Ffig,IfcBeam,Beam rafter,BaseQuantities,Width,71.0
8,11EjrZoVgVGQm8NAB8Ffig,IfcBeam,Beam rafter,BaseQuantities,Depth,171.0
9,11EjrZoVgVGQm8NAB8Ffig,IfcBeam,Beam rafter,BaseQuantities,CrossSectionArea,0.012141


In [ ]:
# 💾 CSV Export: Zusammenfassung
if summary_data:
    summary_rows = [
        {'PropertySet': pset, 'Property': prop}
        for pset, props in summary_data.items()
        for prop in sorted(props)
    ]
    
    df_summary = pd.DataFrame(summary_rows).sort_values(['PropertySet', 'Property']).reset_index(drop=True)
    output_file = 'property_sets_summary.csv'
    df_summary.to_csv(output_file, index=False, encoding='utf-8-sig')
    
    print(f"✅ CSV gespeichert: {output_file}")
    print(f"   Zeilen: {len(df_summary):,}")
    display(df_summary.head(20))
    
    # Download in Colab
    try:
        from google.colab import files
        files.download(output_file)
        print(f"📥 Download gestartet: {output_file}")
    except:
        pass


✅ CSV gespeichert: property_sets_summary.csv
   Zeilen: 106


,PropertySet,Property
0,BaseQuantities,Area
1,BaseQuantities,CrossSectionArea
2,BaseQuantities,Depth
3,BaseQuantities,GrossArea
4,BaseQuantities,GrossFootprintArea
5,BaseQuantities,GrossSideArea
6,BaseQuantities,GrossSurfaceArea
7,BaseQuantities,GrossVolume
8,BaseQuantities,Height
9,BaseQuantities,Length


In [ ]:
# 📊 Statistiken
print("📊 Top 10 PropertySets:")
pset_counts = df_summary.groupby('PropertySet').size().sort_values(ascending=False)
for pset, count in pset_counts.head(10).items():
    print(f"   {count:3d} Properties - {pset}")

print("\n🏗️ Top 10 Element-Klassen:")
element_counts = df_detailed['Element_Class'].value_counts()
for elem_type, count in element_counts.head(10).items():
    print(f"   {count:4d} Properties - {elem_type}")


📊 Top 10 PropertySets:
    20 Properties - BaseQuantities
    14 Properties - Material
    10 Properties - Door Lining Properties
     6 Properties - Pset_DoorCommon
     5 Properties - Pset_CurtainWallCommon
     5 Properties - Pset_DoorWindowGlazingType
     5 Properties - Pset_WindowCommon
     4 Properties - Door Panel Properties - Panel 1
     4 Properties - Door Panel Properties - Panel 2
     4 Properties - Pset_BeamCommon

🏗️ Top 10 Element-Typen:
   2942 Properties - IfcWindow
   2478 Properties - IfcDoor
   2231 Properties - IfcWall
   1601 Properties - IfcCurtainWall
    698 Properties - IfcBeam
    484 Properties - IfcCovering
    252 Properties - IfcColumn
    237 Properties - IfcFurnishingElement
    102 Properties - IfcSlab
     94 Properties - IfcFlowTerminal


## 📈 Interaktives PropertySet-Dashboard

Erweitere den CSV-Export um ein eingebettetes Dash-Dashboard, das PropertySets 
interaktiv visualisiert. Führe die Zellen oben zuerst aus, damit `df_detailed`
bereitsteht.


In [ ]:
# 📦 Dash installieren (nur beim ersten Ausführen erforderlich)
%pip install -q dash>=2.11 plotly


In [ ]:
# 📚 Dash- und Plotly-Imports
from dash import Dash, dcc, html, Input, Output
import plotly.express as px


In [ ]:
# 🚀 Enhanced Multi-Tab Dashboard
if 'df_detailed' not in globals() or df_detailed.empty:
    raise ValueError('df_detailed ist leer. Bitte führe zuerst die Extraktionszellen aus.')

# 📊 Data Preparation for Multiple Visualizations
print("🔧 Bereite Daten für verschiedene Visualisierungen vor...")

# Parse numeric values for quantitative analysis
df_numeric = df_detailed.copy()
df_numeric['Numeric_Value'] = pd.to_numeric(df_numeric['Value'], errors='coerce')
numeric_properties = df_numeric.dropna(subset=['Numeric_Value'])
# Bubble plots cannot handle negative sizes; use absolute magnitude while preserving sign in hover
numeric_properties['Bubble_Size'] = numeric_properties['Numeric_Value'].abs()

# Create aggregations for different chart types
agg_dashboard = (
    df_detailed.groupby(['PropertySet', 'Property'])
    .size()
    .reset_index(name='Count')
    .sort_values(['PropertySet', 'Count'], ascending=[True, False])
)

# Hierarchical data for sunburst
hierarchical_data = (
    df_detailed.groupby(['Element_Class', 'PropertySet', 'Property'])
    .size()
    .reset_index(name='Count')
)

# Element Class × PropertySet pivot for heatmap
heatmap_data = (
    df_detailed.groupby(['Element_Class', 'PropertySet'])
    .size()
    .unstack(fill_value=0)
)

# PropertySet totals for pie chart
pset_totals = df_detailed['PropertySet'].value_counts().reset_index()
pset_totals.columns = ['PropertySet', 'Count']

# Top properties across all PropertySets
top_properties = (
    df_detailed.groupby('Property')
    .size()
    .sort_values(ascending=False)
    .head(15)
    .reset_index()
)
top_properties.columns = ['Property', 'Count']

# Element Class distribution
element_dist = df_detailed['Element_Class'].value_counts().reset_index()
element_dist.columns = ['Element_Class', 'Count']

print(f"✅ Daten vorbereitet: {len(hierarchical_data)} hierarchische Einträge, {len(numeric_properties)} numerische Werte")

# 🎨 Dashboard App
app = Dash(__name__)
app.title = 'IFC PropertySet Analytics Dashboard'

# 📊 Key Metrics Cards
def create_metric_card(title, value, subtitle=""):
    return html.Div([
        html.H3(value, style={'margin': '0', 'font-size': '2rem', 'color': '#1f77b4'}),
        html.P(title, style={'margin': '0', 'font-size': '0.9rem', 'color': '#666'}),
        html.P(subtitle, style={'margin': '0', 'font-size': '0.8rem', 'color': '#999'})
    ], style={
        'background': 'white',
        'padding': '20px',
        'border-radius': '8px',
        'box-shadow': '0 2px 4px rgba(0,0,0,0.1)',
        'text-align': 'center'
    })

# 🎯 Overview Tab Layout
overview_tab = html.Div([
    html.H3('📊 Dashboard Overview', style={'text-align': 'center', 'margin-bottom': '30px'}),
    
    # Metrics Cards Row
    html.Div([
        create_metric_card('Total Properties', f"{len(df_detailed):,}", 'Property-Werte'),
        create_metric_card('PropertySets', f"{len(df_detailed['PropertySet'].unique())}", 'Eindeutige Sets'),
        create_metric_card('Element Classes', f"{len(df_detailed['Element_Class'].unique())}", 'IFC-Typen'),
        create_metric_card('Elements', f"{len(df_detailed['Element_GlobalId'].unique())}", 'Einzelne Elemente')
    ], style={'display': 'grid', 'grid-template-columns': 'repeat(4, 1fr)', 'gap': '20px', 'margin-bottom': '30px'}),
    
    # Treemap
    html.H4('Element Class Distribution'),
    dcc.Graph(
        figure=px.treemap(
            element_dist,
            path=['Element_Class'],
            values='Count',
            title='Element Classes by Property Count',
            color='Count',
            color_continuous_scale='Blues'
        ).update_layout(height=400, margin=dict(t=50, b=20, l=20, r=20))
    )
])

# 🌳 Hierarchical Tab Layout
hierarchical_tab = html.Div([
    html.H3('🌳 Hierarchical Property Structure', style={'text-align': 'center', 'margin-bottom': '30px'}),
    html.P('Element Class → PropertySet → Property relationships', style={'text-align': 'center', 'color': '#666'}),
    
    dcc.Graph(
        figure=px.sunburst(
            hierarchical_data,
            path=['Element_Class', 'PropertySet', 'Property'],
            values='Count',
            title='Property Hierarchy Visualization',
            color='Count',
            color_continuous_scale='Viridis'
        ).update_layout(height=600, margin=dict(t=50, b=20, l=20, r=20))
    )
])

# 📈 Distribution Tab Layout
distribution_tab = html.Div([
    html.H3('📈 Property Distribution Analysis', style={'text-align': 'center', 'margin-bottom': '30px'}),
    
    html.Div([
        # Pie Chart
        html.Div([
            html.H4('PropertySet Distribution'),
            dcc.Graph(
                figure=px.pie(
                    pset_totals.head(10),
                    values='Count',
                    names='PropertySet',
                    title='Top 10 PropertySets by Count',
                    color_discrete_sequence=px.colors.qualitative.Set3
                ).update_layout(height=400, margin=dict(t=50, b=20, l=20, r=20))
            )
        ], style={'width': '48%', 'display': 'inline-block'}),
        
        # Bar Chart
        html.Div([
            html.H4('Most Common Properties'),
            dcc.Graph(
                figure=px.bar(
                    top_properties,
                    x='Count',
                    y='Property',
                    orientation='h',
                    title='Top 15 Properties Across All PropertySets',
                    color='Count',
                    color_continuous_scale='Blues'
                ).update_layout(height=400, margin=dict(t=50, b=20, l=20, r=20))
            )
        ], style={'width': '48%', 'display': 'inline-block', 'margin-left': '4%'})
    ])
])

# 🔥 Element Analysis Tab Layout
element_analysis_tab = html.Div([
    html.H3('🔥 Element Analysis', style={'text-align': 'center', 'margin-bottom': '30px'}),
    
    # Heatmap
    html.H4('PropertySet Coverage by Element Class'),
    dcc.Graph(
        figure=px.imshow(
            heatmap_data.values,
            x=heatmap_data.columns,
            y=heatmap_data.index,
            color_continuous_scale='Blues',
            title='PropertySet Presence Heatmap',
            labels=dict(x='PropertySet', y='Element Class', color='Count')
        ).update_layout(height=500, margin=dict(t=50, b=20, l=20, r=20))
    ),
    
    # Stacked Bar Chart
    html.H4('Properties per Element Class'),
    dcc.Graph(
        figure=px.bar(
            df_detailed.groupby(['Element_Class', 'PropertySet']).size().reset_index(name='Count'),
            x='Element_Class',
            y='Count',
            color='PropertySet',
            title='Property Distribution by Element Class',
            color_discrete_sequence=px.colors.qualitative.Set3
        ).update_layout(height=400, margin=dict(t=50, b=20, l=20, r=20), xaxis_tickangle=-45)
    )
])

# 📊 Numeric Analysis Tab Layout
numeric_analysis_tab = html.Div([
    html.H3('📊 Numeric Property Analysis', style={'text-align': 'center', 'margin-bottom': '30px'}),
    
    html.Div([
        html.P(f"Found {len(numeric_properties)} numeric property values across {len(numeric_properties['Property'].unique())} properties"),
        
        # Box Plot
        html.H4('Numeric Property Distribution'),
        dcc.Graph(
            figure=px.box(
                numeric_properties,
                x='Element_Class',
                y='Numeric_Value',
                color='Element_Class',
                title='Numeric Property Values by Element Class',
                color_discrete_sequence=px.colors.qualitative.Set3
            ).update_layout(height=400, margin=dict(t=50, b=20, l=20, r=20), xaxis_tickangle=-45)
        ),
        
        # Scatter Plot (if we have enough numeric data)
        html.H4('Property Value Relationships') if len(numeric_properties) > 10 else html.Div(),
        dcc.Graph(
            figure=px.scatter(
                numeric_properties,
                x='Property',
                y='Numeric_Value',
                color='Element_Class',
                size='Bubble_Size',
                title='Numeric Property Values Distribution',
                color_discrete_sequence=px.colors.qualitative.Set3
            ).update_layout(height=400, margin=dict(t=50, b=20, l=20, r=20), xaxis_tickangle=-45)
        ) if len(numeric_properties) > 10 else html.Div('Insufficient numeric data for scatter plot')
    ])
])

# 📋 Details Tab Layout
details_tab = html.Div([
    html.H3('📋 Detailed Property Data', style={'text-align': 'center', 'margin-bottom': '30px'}),
    
    html.Div([
        html.Label('PropertySet auswählen:'),
        dcc.Dropdown(
            id='pset-dropdown',
            options=[{'label': pset, 'value': pset} for pset in df_detailed['PropertySet'].unique()],
            value=df_detailed['PropertySet'].unique()[0],
            clearable=False,
        ),
    ], style={'width': '40%', 'margin-bottom': '20px'}),
    
    html.Div(id='pset-detail-table')
])

# 🎨 Main App Layout
app.layout = html.Div([
    html.H1('IFC PropertySet Analytics Dashboard', 
             style={'text-align': 'center', 'color': '#1f77b4', 'margin-bottom': '30px'}),
    
    dcc.Tabs(id='main-tabs', children=[
        dcc.Tab(label='📊 Overview', children=overview_tab),
        dcc.Tab(label='🌳 Hierarchical', children=hierarchical_tab),
        dcc.Tab(label='📈 Distribution', children=distribution_tab),
        dcc.Tab(label='🔥 Element Analysis', children=element_analysis_tab),
        dcc.Tab(label='📊 Numeric Analysis', children=numeric_analysis_tab),
        dcc.Tab(label='📋 Details', children=details_tab)
    ], style={'margin-bottom': '20px'})
], style={'maxWidth': '1400px', 'margin': '0 auto', 'padding': '20px', 'font-family': 'Arial, sans-serif'})

# 🔄 Callback for Details Tab
@app.callback(
    Output('pset-detail-table', 'children'),
    Input('pset-dropdown', 'value')
)
def update_detail_table(selected_pset):
    subset = df_detailed[df_detailed['PropertySet'] == selected_pset]
    if subset.empty:
        return html.Div('Keine Einträge für dieses PropertySet gefunden.')
    
    top_rows = subset.head(25)
    header = html.Tr([html.Th(col, style={'padding': '8px', 'border': '1px solid #ddd'}) for col in top_rows.columns])
    body = [
        html.Tr([html.Td(str(top_rows.iloc[i][col]), style={'padding': '8px', 'border': '1px solid #ddd'}) 
                 for col in top_rows.columns])
        for i in range(len(top_rows))
    ]
    
    return html.Table(
        [header] + body, 
        style={'width': '100%', 'border-collapse': 'collapse', 'margin-top': '20px'}
    )

print("🚀 Enhanced Dashboard gestartet!")
app.run(jupyter_mode='inline', jupyter_height=800, jupyter_width='100%')
